## Code

The code here correspond to the Algorithm 1 in Appendix G of the paper, and uses an iterative algorithm that is akin to policy-iteration described in Appendix G.

### 1 Simulation Class

We start with initializing our simulation class. This class takes the following inputs:

* _epsilon_: The horizontal differentiation parameter $\epsilon$.
* *lambda_S*: The arrival rate $\lambda_S$ for suppliers.
* *lambda_D* : The arrival rate $\lambda_C$ for customers.
* _c_: The list of discretized supplier costs ${\cal T}$ defined in Appendix G.
* _v_: The list of discretized customer valuations ${\cal T}$ defined in Appendix G.
* *beta_a_demand, beta_b_demand*: $\alpha$ and $\beta$ parameters for the Beta arrival distribution of customers.
* *beta_a_supply, beta_b_supply*: $\alpha$ and $\beta$ parameters for the Beta arrival distribution of suppliers.
* _alpha_: alpha is such that the platform's objective function is $alpha \cdot \text{social welfare}+(1-alpha) \cdot \text{platform's gross revenue}$.
* *iteration_nu*: Number of high-level iterations, defined as $I$ in Appendix G of the paper, that is used in finding the best-response strategies.

In [362]:
import numpy as np
import pandas as pd
from scipy.stats import beta
import os as os
import matplotlib.pyplot as plt
#!pip install ipython

class simul(object):
    def __init__(self,epsilon,c,v,lambda_S,lambda_D,mu_S,mu_D,beta_a_demand=1,beta_b_demand=1\
                 ,beta_a_supply=1,beta_b_supply=1,alpha=1,\
                 iteration_nu=10,trace_convergence=0.01):
        self.eps=epsilon
        self.c=np.array(c)
        self.v=np.array(v)
        self.n=len(self.c)
        ##Simulation threshold is 10**-5 for n=100
        self.simulation_threshold=0.001/(self.n)
        self.lambda_S=lambda_S
        self.lambda_D=lambda_D
        self.mu_S=np.array(mu_S)
        self.mu_D=np.array(mu_D)
        #initialize prices: tau_S represents the vector of prices for each type of supplier
        self.tau_S=self.c+self.eps*0.1
        self.tau_S[self.tau_S>1+self.eps]=1+self.eps
        #initialize thresholds tau_D
        self.tau_D=self.v*0.2
        #initialize the mass of supply and demand as 0 for each discrete type
        self.N_S=np.repeat(0,self.n)
        self.N_D=np.repeat(0,self.n)
        #store initial strategies in separate vectors
        self.initial_tau_S=np.copy(self.tau_S)
        self.initial_tau_D=np.copy(self.tau_D)
        #initialize alpha
        self.alpha=alpha
        #initialized alpha and beta parameters of demand and supply
        self.beta_a_supply=beta_a_supply
        self.beta_b_supply=beta_b_supply
        self.beta_a_demand=beta_a_demand
        self.beta_b_demand=beta_b_demand
        #initialize the number of iterations I used in computing best-response strategies
        self.iteration_nu=iteration_nu
        self.pmin=0
        self.pmax=1+epsilon
        

Next, we add the method *match* that performs the matching process. 

In [363]:
def match(self): 
    # Determine which side is the short one (idx=0 --> supply, idx=1 --> demand)
    total_supply, total_demand = np.sum(self.N_S), np.sum(self.N_D)
    short_side_val, short_side_idx = min((val, idx) for idx, val in enumerate([total_supply, total_demand]))

    # Compute utility for demand and supply
    # x_ij represents customer i's utility from matching supplier j
    x = (np.repeat([self.tau_D], len(self.tau_S), axis=0).transpose() -
         np.repeat([self.v], len(self.tau_S), axis=0).transpose() +
         np.repeat([self.tau_S], len(self.v), axis=0))

    # Compute the proportion of willing participants
    proportion_willing = (self.eps - x) / (2 * self.eps)
    proportion_willing = np.clip(proportion_willing, 0, 1)

    # Compute utility for supply
    util_supply = self.tau_S - self.c
    util_supply = np.maximum(util_supply, 0)

    # If supply utility is <= 0, proportion_willing must be zero 
    proportion_willing *= util_supply > 0

    # Compute expected utility for (demand, supply) type-pairs willing to match each other
    expected_utility = ((np.repeat([self.tau_D], len(self.tau_S), axis=0).transpose() +
                         np.repeat([self.v], len(self.tau_S), axis=0).transpose() -
                         np.repeat([self.tau_S], len(self.v), axis=0) + self.eps) / 2)

    # Update expected utility for cases with proportion_willing equal to 1
    mask = proportion_willing == 1
    expected_utility[mask] = (np.repeat([self.v], len(self.tau_S), axis=0).transpose()[mask] -
                              np.repeat([self.tau_S], len(self.v), axis=0)[mask])

    # Compute the proportions of supply and demand by types
    prop_D = self.N_D / np.sum(self.N_D)
    prop_D.shape = (len(prop_D), 1)

    prop_S = np.zeros(len(self.N_S)) if np.sum(self.N_S) == 0 else self.N_S / np.sum(self.N_S)

    # Compute the number of matches for each demand-supply pair
    num_matched = (total_supply if short_side_idx == 0 else total_demand) * proportion_willing * prop_S * prop_D

    # Compute total number of matches for demand and supply
    self.total_matched_supply = np.sum(num_matched, axis=0)
    self.total_matched_demand = np.sum(num_matched, axis=1)

    # Compute utilities for the time step
    self.util_demand_instant = np.sum(expected_utility * num_matched, axis=1)
    self.util_supply_instant = util_supply * self.total_matched_supply

    # Compute agent surplus, platform gross revenue, and total welfare
    self.agent_surplus = self.util_demand_instant + self.util_supply_instant
    self.platform_utility = np.sum(self.tau_S * self.total_matched_supply)
    self.total_welfare = np.sum(self.util_demand_instant) + np.sum(self.util_supply_instant)

    # Compute the objective function
    self.objective = self.alpha * self.total_welfare + (1 - self.alpha) * self.platform_utility

simul.match=match

Now that we have the match method, we define a new method called simulate that simulates a dynamic market given the strategies.

In [364]:
def simulate(self):
    old_N_S = np.repeat(-9999,self.n)
    old_N_D = np.repeat(-9999,self.n)
    
    # Continue simulation until the difference between old and new states is below the threshold
    while np.nansum(np.abs(self.N_S - old_N_S)) + np.nansum(np.abs(self.N_D - old_N_D)) > self.simulation_threshold:
        old_N_S = self.N_S.copy()
        old_N_D = self.N_D.copy()

        # Entry decision vectors for demand and supply
        # Demand side: Enter if there's a chance of positive utility
        entry_D = (self.v + self.eps > self.tau_D) if len(self.tau_S[self.tau_S >= self.c]) == 0 else (self.v - np.nanmin(self.tau_S[self.tau_S >= self.c]) + self.eps > self.tau_D)

        # Supply side: Two conditions must be met for entry
        entry_S_1 = (self.tau_S - self.eps) < np.nanmax(self.v - self.tau_D)  # Some customers are willing to pay the price
        entry_S_2 = self.tau_S > self.c  # Price higher than cost

        # Distribution of agents entering the market
        arrival_D = self.lambda_D * entry_D
        arrival_S = self.lambda_S * entry_S_1 * entry_S_2

        # Add arrivals with beta-distributed preference, adjusting the scale
        beta_D = beta.pdf(self.v - 1 / (2 * len(self.v)), self.beta_a_demand, self.beta_b_demand, -self.eps, 1 + self.eps)
        beta_D /= np.nanmean(beta_D)

        beta_S = beta.pdf(self.c - 1 / (2 * len(self.c)), self.beta_a_supply, self.beta_b_supply, 0, 1 + self.eps)
        beta_S /= np.nanmean(beta_S)

        self.N_S = self.N_S + arrival_S * beta_S
        self.N_D = self.N_D + arrival_D * beta_D

        # Perform matching
        self.match()

        # Update the number of agents by subtracting matched agents and considering departures
        self.N_S = (self.N_S - self.total_matched_supply) * (1 - self.mu_S)
        self.N_D = (self.N_D - self.total_matched_demand) * (1 - self.mu_D)
        
simul.simulate=simulate

The following method employs a greedy sequential optimization procedure for to calculate the best-response strategies for the decentralized setting where there are no constrained on the prices.

In [365]:
def optimize_decentralized(self, increment=0.01, iteration_nu=10):
    # Initialize minimum and maximum prices
    self.p_min = 0
    self.p_max = 1 + self.eps

    # Initialize max_tau and max_tau_S
    max_tau = -999
    max_tau_S = -999

    # Loop through the specified number of iterations
    for it in range(iteration_nu):
        # Iterate through buyer types
        for j in range(self.n):
            # Initialize maximum utility
            max_util = -999
            max_range_increment = int(np.floor((self.v[j] + self.eps - max(0, np.min(self.tau_S))) / increment))
            
            # Iterate through thresholds to find the utility-maximizing threshold
            for i in range(max_range_increment + 1):
                self.tau_D[j] = increment * i
                self.simulate()
                if self.util_demand_instant[j] > max_util:
                    max_util = np.copy(self.util_demand_instant[j])
                    max_tau = increment * i

            # Update the threshold for the current buyer
            self.tau_D[j] = np.copy(max_tau)
            
        # Iterate through seller types
        for j in reversed(range(self.n)):
            # Calculate maximum allowed price
            max_price = min(self.p_max, np.max(self.v - self.tau_D + self.eps))
            max_range_increment_price = int(np.floor(max_price / increment))
            max_util = -999

            # Calculate minimum increment value such that price is greater than cost and pmin
            min_range_increment = int(max(self.pmin, np.ceil(self.c[j] / increment)))

            # Iterate through prices to find the profit-maximizing price
            for i in range(min(max_range_increment_price, min_range_increment), max_range_increment_price + 1):
                self.tau_S[j] = increment * i + increment * 0.1
                self.simulate()
    
                if self.util_supply_instant[j] > max_util:
                    max_util = np.copy(self.util_supply_instant[j])
                    max_tau_S = increment * i + increment * 0.1

            # Update the price for the current seller
            self.tau_S[j] = np.copy(max_tau_S)

        # Uncomment the following two lines to see that prices and thresholds converge
        # print("Prices: ", self.tau_S)
        # print("Thresholds: ", self.tau_D)

    # Run the final simulation
    self.simulate()

    # Update display attributes
    sim.thresholds = sim.tau_D
    sim.prices = sim.tau_S
    sim.thresholds[sim.util_demand_instant == 0] = float('nan')
    sim.prices[sim.util_supply_instant == 0] = float('nan')

# Assign the optimize_decentralized method to the simul object
simul.optimize_decentralized = optimize_decentralized


Next, we introduce the method to calculate best-response strategies in a centralized setting. This code uses the same procedure as optimize_decentralized.

In [366]:
def optimize_centralized(self, supply_increment=0.02, increment=0.01, iteration_nu=10):
    range_of_increments_supply = int(np.floor((1 + self.eps) / supply_increment))
    util = -999
    max_tau=-999
    # Iterate through possible single prices
    for i in range(range_of_increments_supply + 1):
        current_price = supply_increment * i + supply_increment / 2
        self.tau_S = current_price*np.ones(self.n)
        
        # Sub-procedure: find optimal buyer thresholds
        for it in range(iteration_nu):
            
            for j in range(self.n):
                max_util = -999
                max_range_increment = int(np.floor((self.v[j] + self.eps - max(0, np.min(self.tau_S))) / increment))

                for inner_i in range(max_range_increment + 1):
                    self.tau_D[j] = increment * inner_i
                    self.simulate()
                    if self.util_demand_instant[j] > max_util:
                        max_util = np.copy(self.util_demand_instant[j])
                        max_tau = increment * inner_i

                self.tau_D[j] = np.copy(max_tau)
        self.simulate()
        # Update the best objective value and the corresponding price
        if self.objective > util:
            util = self.objective
            best_prices = current_price*np.ones(self.n)
    
    self.tau_S=best_prices
    # Simulate with the best price and best responses
    self.simulate()

simul.optimize_centralized = optimize_centralized

Method for reporting the results:

In [367]:
def report_OPT(self,setting):
    instant_dict={'v':self.v,'c':self.c,'v_minus_tauD':self.v-self.tau_D,'tau_D':self.tau_D,'tau_S':self.tau_S,\
    'lambda_S':self.lambda_S,'lambda_D':self.lambda_D,'mu_S':np.repeat(self.mu_S,self.n),'mu_D':np.repeat(self.mu_D,self.n),'epsilon':np.repeat(self.eps,self.n),\
       'N_D':self.N_D_just_before,'N_S':self.N_S_just_before,'Total_Welfare':np.repeat(self.total_welfare,self.n),\
       'Supply_Revenue':self.util_supply_instant,'Buyer_Welfare':self.util_demand_instant,\
       'Platform_Revenue':np.repeat(self.platform_utility,self.n),\
       'Setting':np.repeat(setting,self.n),'alpha':self.alpha,'iteration_nu_max':self.iteration_nu_max,\
       'Initial_tauD':self.initial_tau_D,'Initial_tauS':self.initial_tau_S,\
       'beta_a_demand':self.beta_a_demand,'beta_b_demand':self.beta_b_demand,\
       'beta_a_supply':self.beta_a_supply,'beta_b_supply':self.beta_b_supply}
    self.instant_df=pd.DataFrame(data=instant_dict)

### Sample run for number of types n=20

In [368]:
##Number of types
n=20
##set epsilon
eps=0.1
#set departures rates for supply and demand
mu_S,mu_D=0.1,0.1
#set arrival rates for supply and demand
lambda_S,lambda_D=1,1
#set costs and valuations of agents 
c,v=np.array(range(n))*((1+eps)/n)+(1+eps)/n,\
    np.array(range(n))*((1+eps)/n)+(1+eps)/n-eps
#create the simulation object
sim=simul(eps,c,v,lambda_S,lambda_D,mu_S,mu_D)
#find decentralized equilibrium
sim.optimize_decentralized()
#Report the results
print("Prices:",sim.prices)
print("Thresholds:",sim.thresholds)
print("Supplier utilities:", sim.util_supply_instant)
print("Customer utilities:", sim.util_demand_instant)

Prices: [0.531 0.531 0.541 0.561 0.571 0.581 0.591 0.611 0.631 0.641 0.661 0.691
 0.731   nan   nan   nan   nan   nan   nan   nan]
Thresholds: [ nan  nan  nan  nan  nan  nan  nan  nan  nan 0.   0.01 0.04 0.07 0.1
 0.14 0.18 0.22 0.27 0.31 0.35]
Supplier utilities: [0.40086275 0.35454458 0.30916287 0.26504178 0.22259379 0.18138119
 0.1417341  0.10504769 0.07193484 0.04371934 0.02118084 0.00635657
 0.00048822 0.         0.         0.         0.         0.
 0.         0.        ]
Customer utilities: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.00074138 0.01286239 0.03880747
 0.0731839  0.11292231 0.15604354 0.20143376 0.24855388 0.29706347
 0.34654944 0.39668728]


Here, we note the nan values of strategies correspond to types that do not enter the market. We observe that utilities correspond to those who do not enter are 0.

### 2 Run Class

In this section, we introduce a new class called *run class* to facilitate running simulations for various market conditions. 

In [20]:
class runclass(object):
    def __init__(self,epsilon,n,lamS_vector,lamD_vector,muS_vector,alpha_vector,\
                 iteration_nu,beta_a_demand,beta_b_demand,beta_a_supply,beta_b_supply,\
                 demand_increment=0.01,supply_increment=0.02):
        self.run_length=len(lamS_vector)
        self.n=n
        self.eps=epsilon
        self.lamS_vector=np.array(lamS_vector)
        self.lamD_vector=np.array(lamD_vector)
        self.muS_vector=np.array(muS_vector)
        self.muD_vector=np.array(muS_vector)
        self.iteration_nu=iteration_nu
        self.demand_increment=demand_increment
        self.supply_increment=supply_increment
        self.alpha_vector=np.array(alpha_vector)
        self.beta_a_supply=beta_a_supply
        self.beta_b_supply=beta_b_supply
        self.beta_a_demand=beta_a_demand
        self.beta_b_demand=beta_b_demand

Next, we add the method for running the decentralized setting into the class runclass.

In [21]:
def run_decentralized(self):

    df_list=[]

    settingD=['Decentralized']

    for lam_S in self.lamS_vector:
        for lam_D in self.lamD_vector:
            for muS in self.muS_vector:
                #for muD in self.muD_vector:
                    for eps in self.eps:
                        muD=muS

                        self.c=np.array(range(self.n))*((1+eps)/self.n)+(1+eps)/self.n
                        self.v=np.array(range(self.n))*((1+eps)/self.n)+(1+eps)/self.n-eps

                        start=time.time()  ##trace total time to run                          
                        print(eps,self.c,self.v,lam_S,lam_D,muS,muD,\
                                       self.beta_a_demand,self.beta_b_demand,\
                                       self.beta_a_supply,self.beta_b_supply)

                        simD=simul(eps,self.c,self.v,lam_S,lam_D,muS,muD,\
                                       self.beta_a_demand,self.beta_b_demand,\
                                       self.beta_a_supply,self.beta_b_supply)

                        simD.optimize_decentralized(self.demand_increment)

                        simD.report_OPT(setting=settingD)                    
                        df_list.append(simD.instant_df)

                        end=time.time()

    self.df_final=pd.concat(df_list)
    self.df_final.reset_index(inplace=True)
    
    path='ResultsDecentralized_'+'_n_is_'+str(self.n)+'_muSD_'+str(self.muS_vector)+str(self.muD_vector)\
            +'_lambdaSD_'+str(self.lamS_vector)+str(self.lamD_vector)\
            +'_eps_'+str(self.eps)+'_alpha_'+str(len(self.alpha_vector))+\
            '_betasSD_'+str(self.beta_a_supply)+str(self.beta_b_supply)+str(self.beta_a_demand)+str(self.beta_b_demand)+'.csv'
    
    self.df_final.to_csv(path)  
    
runclass.run_decentralized=run_decentralized

Appended cell to runclass


Similarly, we add the method for running the centralized simulation.

In [22]:
def run_centralized(self):

    df_list=[]

    settingC=['Centralized']

    for alpha in self.alpha_vector:                                        
        for lam_S in self.lamS_vector:
            for lam_D in self.lamD_vector:
                for muS in self.muS_vector:
                    #for muD in self.muD_vector:
                        for eps in self.eps:
                            muD=muS
                            start=time.time()

                            self.c=np.array(range(self.n))*((1+eps)/self.n)+(1+eps)/self.n
                            self.v=np.array(range(self.n))*((1+eps)/self.n)+(1+eps)/self.n-eps

                            print(eps,self.c,self.v,lam_S,lam_D,muS,muD,\
                                       self.beta_a_demand,self.beta_b_demand,\
                                       self.beta_a_supply,self.beta_b_supply,\
                                       alpha)
                            simC=simul(eps,self.c,self.v,lam_S,lam_D,muS,muD,\
                                       self.beta_a_demand,self.beta_b_demand,\
                                       self.beta_a_supply,self.beta_b_supply,\
                                       alpha=alpha)

                            simC.optimize_centralized(self.supply_increment)

                            simC.report_OPT(setting=settingC)
                            df_list.append(simC.instant_df)

                            end=time.time()                              
                            print("Total run time: ",end-start)

    self.df_final=pd.concat(df_list)
    self.df_final.reset_index(inplace=True)
    
    ##create the path file
    path='ResultsCentralized_'+'_n_is_'+str(self.n)+'_muSD_'+str(self.muS_vector)+str(self.muD_vector)\
            +'_lambdaSD_'+str(self.lamS_vector)+str(self.lamD_vector)\
            +'_eps_'+str(self.eps)+'_alpha_'+str(len(self.alpha_vector))+\
            '_betasSD_'+str(self.beta_a_supply)+str(self.beta_b_supply)+str(self.beta_a_demand)+str(self.beta_b_demand)+'.csv'
    
    self.df_final.to_csv(path)  

runclass.run_centralized=run_centralized

Appended cell to runclass


### 3 Run code to obtain the data used in the paper

The following code creates an object of runclass defined in Section 2 and simulates the centralized and the decentralized setting for given sets of market conditions. 

In [ ]:
##Number of iterations to use in finding best-response strategies
iteration_nu=10
##Number of discretized types
n=100
##Vector of epsilons to try
epsilon=[0.5,1]
##1 for welfare-maximizing platform, 0 for revenue maximizing platform
alpha_vector=[1] 
## alpha and beta parameters of Beta distributed arrivals
beta_a_demand=1; beta_b_demand=1; beta_a_supply=1 ; beta_b_supply=1;
## Arrival rates for supply and demand(customers)
lamS_vector=[100]
lamD_vector=[10,50,200,1000]                    
## Departure rates
mu_vector=[0.1]      

run=runclass(epsilon,n,lamS_vector,lamD_vector,mu_vector,\
                 alpha_vector,iteration_nu,beta_a_demand,beta_b_demand,\
                 beta_a_supply,beta_b_supply,supply_increment=0.01,demand_increment=0.01)

run.run_centralized()
run.run_decentralized()